In [51]:
# https://stackoverflow.com/questions/65227103/clause-extraction-long-sentence-segmentation-in-python


import spacy
import deplacy
from nltk import Tree
en = spacy.load('en_core_web_md')

#text = "This all encompassing experience wore off for a moment and in that moment, my awareness came gasping to the surface of the hallucination and I was able to consider momentarily that I had killed myself by taking an outrageous dose of an online drug and this was the most pathetic death experience of all time."
text = "Automated inserting and categorizing 20k+ emails from over 200 clients in a CRM by creating an email monitoring REST API using Express, and deployed the service on Azure Cloud with Docker."
#text = "he plays cricket but does not play hockey."

doc = en(text)
#deplacy.render(doc)
def to_nltk_tree(node):
    if node.n_lefts + node.n_rights > 0:
        return Tree(node.orth_, [to_nltk_tree(child) for child in node.children])
    else:
        return node.orth_

[to_nltk_tree(sent.root).pretty_print() for sent in doc.sents]

seen = set() # keep track of covered words

chunks = []
for sent in doc.sents:
    #[print(str(x) + "~") for x in sent.root.children]
    heads = [cc for cc in sent.root.children if cc.dep_ in ['conj', 'adp']]
    print(len(heads))

    for head in heads:
        words = [ww for ww in head.subtree]
        for word in words:
            seen.add(word)
        chunk = (' '.join([ww.text for ww in words]))
        chunks.append( (head.i, chunk) )

    # this last piece of code is a really lazy way to print the remaining chars and
    # saying that it's a clause, when in fact it isn't
    #unseen = [ww for ww in sent if ww not in seen]
    #chunk = ' '.join([ww.text for ww in unseen])
    #chunks.append( (sent.root.i, chunk) )

chunks = sorted(chunks, key=lambda x: x[0])

for ii, chunk in chunks:
    print(ii, chunk, end="\n\n")
        

                                 inserting                                                                      
     ________________________________|______________________________________________________________             
    |      |   |   |   |                   categorizing                                             |           
    |      |   |   |   |     ___________________|_____________________                              |            
    |      |   |   |   |    |        |          |                     by                            |           
    |      |   |   |   |    |        |          |                     |                             |            
    |      |   |   |   |    |        |          |                  creating                      deployed       
    |      |   |   |   |    |        |          |              _______|_________             _______|_______     
    |      |   |   |   |    |       from        |           email               |        ser

In [52]:
# https://stackoverflow.com/questions/65227103/clause-extraction-long-sentence-segmentation-in-python


import spacy
import deplacy
from nltk import Tree
en = spacy.load('en_core_web_md')

#text = "This all encompassing experience wore off for a moment and in that moment, my awareness came gasping to the surface of the hallucination and I was able to consider momentarily that I had killed myself by taking an outrageous dose of an online drug and this was the most pathetic death experience of all time."
#text = "Automated inserting and categorizing 20k+ emails from over 200 clients in a CRM by creating an email monitoring REST API using Express, and deployed the service on Azure Cloud with Docker."
text = "he plays cricket but does not play hockey."

doc = en(text)
#deplacy.render(doc)
def to_nltk_tree(node):
    if node.n_lefts + node.n_rights > 0:
        return Tree(node.orth_, [to_nltk_tree(child) for child in node.children])
    else:
        return node.orth_

[to_nltk_tree(sent.root).pretty_print() for sent in doc.sents]

seen = set() # keep track of covered words

chunks = []
for sent in doc.sents:
    #[print(str(x) + "~") for x in sent.root.children]
    heads = [cc for cc in sent.root.children if cc.dep_ in ['conj']]
    print(len(heads))

    for head in heads:
        words = [ww for ww in head.subtree]
        for word in words:
            seen.add(word)
        chunk = (' '.join([ww.text for ww in words]))
        chunks.append( (head.i, chunk) )

chunks = sorted(chunks, key=lambda x: x[0])

for ii, chunk in chunks:
    print(ii, chunk, end="\n\n")
        

                plays                 
  ________________|_________           
 |     |     |    |        play       
 |     |     |    |     ____|_____     
 he cricket but   .   does not  hockey

1
6 does not play hockey



In [28]:
spacy.explain("ADP")

'adposition'

### Stanford Core NLP
download from https://stanfordnlp.github.io/CoreNLP/

then run 

`cd /Users/curtis/Desktop/idioms/pkg/stanford-corenlp-4.1.0;
java -mx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 15000`

In [2]:
# from https://github.com/rahulkg31/sentence-to-clauses/blob/master/sent_to_clauses.py
import nltk
import re
from nltk.tree import ParentedTree
from pycorenlp import *
# start the connection here
nlp=StanfordCoreNLP("http://0.0.0.0:9000/")
import json

In [16]:
# get verb phrases
# if one "VP" node has 2 or more "VP" children then
# all child "VP" while be used as verb phrases
# since a clause may have more than one verb phrases
# ex:- he plays cricket but does not play hockey
# here two verb phrases are "plays cricket" and "does not play hockey"
#                       ROOT
#                        |
#                        S
#   _____________________|____
#  |                          VP
#  |          ________________|____
#  |         |           |         VP
#  |         |           |     ____|________
#  |         VP          |    |    |        VP
#  |     ____|_____      |    |    |    ____|____
#  NP   |          NP    |    |    |   |         NP
#  |    |          |     |    |    |   |         |
# PRP  VBZ         NN    CC  VBZ   RB  VB        NN
#  |    |          |     |    |    |   |         |
# 1
def get_verb_phrases(t):
    verb_phrases = []
    num_children = len(t)
    num_VP = sum(1 if t[i].label() == "VP" else 0 for i in range(0, num_children))

    if t.label() != "VP":
        for i in range(0, num_children):
            if t[i].height() > 2:
                verb_phrases.extend(get_verb_phrases(t[i]))
    elif t.label() == "VP" and num_VP > 1:
        for i in range(0, num_children):
            if t[i].label() == "VP":
                if t[i].height() > 2:
                    verb_phrases.extend(get_verb_phrases(t[i]))
    else:
        verb_phrases.append(' '.join(t.leaves()))

    return verb_phrases

# get position of first node "VP" while traversing from top to bottom
# get the position of subordinating conjunctions like after, as, before, if, since, while etc
# delete the node at these positions to get the subject
# first delete vp nodes then subordinating conjunction nodes
# ie, get the part without verb phrases
# in the above example "he" will be returned
def get_pos(t):
    vp_pos = []
    sub_conj_pos = []
    num_children = len(t)
    children = [t[i].label() for i in range(0,num_children)]

    flag = re.search(r"(S|SBAR|SBARQ|SINV|SQ)", ' '.join(children))

    if "VP" in children and not flag:
        for i in range(0, num_children):
            if t[i].label() == "VP":
                vp_pos.append(t[i].treeposition())
    elif not "VP" in children and not flag:
        for i in range(0, num_children):
            if t[i].height() > 2:
                temp1,temp2 = get_pos(t[i])
                vp_pos.extend(temp1)
                sub_conj_pos.extend(temp2)
    # comment this "else" part, if want to include subordinating conjunctions
    else:
        for i in range(0, num_children):
            if t[i].label() in ["S","SBAR","SBARQ","SINV","SQ"]:
                temp1, temp2 = get_pos(t[i])
                vp_pos.extend(temp1)
                sub_conj_pos.extend(temp2)
            else:
                sub_conj_pos.append(t[i].treeposition())

    return (vp_pos,sub_conj_pos)


# get all clauses
def get_clause_list(sent):
    parser = nlp.annotate(sent, properties={"annotators":"parse","outputFormat": "json"})
    parser=json.loads(parser)
    #print(parser["sentences"][0]["parse"])
    sent_tree = ParentedTree.fromstring(parser["sentences"][0]["parse"])
    clause_level_list = ["S","SBAR","SBARQ","SINV","SQ"]
    clause_list = []
    sub_trees = []
    # sent_tree.pretty_print()

    # break the tree into subtrees of clauses using
    # clause levels "S","SBAR","SBARQ","SINV","SQ"
    for sub_tree in reversed(list(sent_tree.subtrees())):
        #print(sub_tree.label())
        if sub_tree.label() in clause_level_list:
            if sub_tree.parent().label() in clause_level_list:
                continue
            if (len(sub_tree) == 1 and sub_tree.label() == "S" and sub_tree[0].label() == "VP"
                and not sub_tree.parent().label() in clause_level_list):
                continue

            sub_trees.append(sub_tree)
            del sent_tree[sub_tree.treeposition()]

    # for each clause level subtree, extract relevant simple sentence
    for t in sub_trees:
        # get verb phrases from the new modified tree
        verb_phrases = get_verb_phrases(t)

        # get tree without verb phrases (mainly subject)
        # remove subordinating conjunctions
        vp_pos,sub_conj_pos = get_pos(t)
        for i in vp_pos:
            del t[i]
        for i in sub_conj_pos:
            del t[i]
        #print(t.label())

        # I commented this out so we don't attch the subject to each phrase
        subject_phrase = ""#' '.join(t.leaves())

        # update the clause_list
        for i in verb_phrases:
            clause_list.append(subject_phrase + " " + i)

    clause_list.reverse()
    return clause_list

#if __name__ == "__main__":
    # sent = "he plays cricket but does not play hockey"
    # sent = re.sub(r"(\.|,|\?|\(|\)|\[|\])"," ",sent)
    # clause_qlist = get_clause_list(sent)
    # print(clause_list)
#while (True):
#sent = input("sentence : \n ")
sent = "Automated inserting and categorizing 20k+ emails from over 200 clients in a CRM by creating an email monitoring REST API using Express, and deployed the service on Azure Cloud with Docker."
#sent = "he plays cricket but does not play hockey"
#sent = re.sub(r"(\.|,|\?|\(|\)|\[|\])", " ", sent)
clauses = get_clause_list(sent)
print("num clauses:", len(clauses))
for clause in clauses:
    print("-" + clause,end="\n\n")
#NOTE: sentences must end in a period.

num clauses: 3
- deployed the service on Azure Cloud with Docker

- REST API using Express

- inserting and categorizing 20k emails from over 200 clients in a CRM by creating an email monitoring



In [19]:
        # S = Simple declarative clause
        # SBar = Subordinate Clause
        # SBARQ = Direct question introduced by wh-element
        # SINV = Declarative sentence with subject-aux inversion
        # SQ = Yes/no questions and subconstituent of SBARQ excluding wh-element
sent = "he plays cricket but does not play hockey."
parser = nlp.annotate(sent, properties={"annotators": "parse", "outputFormat": "json"})
parser = json.loads(parser)
sent_tree = ParentedTree.fromstring(parser["sentences"][0]["parse"])
for cur in sent_tree.subtrees():
    if cur.label in ["S","SBAR","SBARQ","SINV","SQ", "PP"]:
        print(cur.leaves())

---------
['he', 'plays', 'cricket', 'but', 'does', 'not', 'play', 'hockey', '.']
---------
['he', 'plays', 'cricket', 'but', 'does', 'not', 'play', 'hockey', '.']
---------
['he']
---------
['he']
---------
['plays', 'cricket', 'but', 'does', 'not', 'play', 'hockey']
---------
['plays', 'cricket']
---------
['plays']
---------
['cricket']
---------
['cricket']
---------
['but']
---------
['does', 'not', 'play', 'hockey']
---------
['does']
---------
['not']
---------
['play', 'hockey']
---------
['play']
---------
['hockey']
---------
['hockey']
---------
['.']


input: sentence
output: clauses

clause:{
    verb phrase:
    subject:
}

In [12]:
parser = nlp.annotate(sent, properties={"annotators":"parse","outputFormat": "json"})
parser=json.loads(parser)
#print(parser["sentences"][0]["parse"])
sent_tree = ParentedTree.fromstring(parser["sentences"][0]["parse"])

In [16]:
def parse_clauses(sent):
    parser = nlp.annotate(sent, properties={"annotators":"parse","outputFormat": "json"})
    parser=json.loads(parser)
    #print(parser["sentences"][0]["parse"])
    sent_tree = ParentedTree.fromstring(parser["sentences"][0]["parse"])
    clause_level_list = ["S","SBAR","SBARQ","SINV","SQ"]
    clause_list = []
    sub_trees = []
    # sent_tree.pretty_print()

    # break the tree into subtrees of clauses using
    # clause levels "S","SBAR","SBARQ","SINV","SQ"
    for sub_tree in reversed(list(sent_tree.subtrees())):
        print(sub_tree)
        print(sub_tree.leaves())
    
    #if "VP" in children and not flag:
    #    for i in range(0, num_children):
    #        if t[i].label() == "VP":
    #            vp_pos.append(t[i].treeposition())
parse_clauses(sent)

(NN hockey)
['hockey']
(NP (NN hockey))
['hockey']
(VB play)
['play']
(VP (VB play) (NP (NN hockey)))
['play', 'hockey']
(RB not)
['not']
(VBZ does)
['does']
(VP (VBZ does) (RB not) (VP (VB play) (NP (NN hockey))))
['does', 'not', 'play', 'hockey']
(CC but)
['but']
(NN cricket)
['cricket']
(NP (NN cricket))
['cricket']
(VBZ plays)
['plays']
(VP (VBZ plays) (NP (NN cricket)))
['plays', 'cricket']
(VP
  (VP (VBZ plays) (NP (NN cricket)))
  (CC but)
  (VP (VBZ does) (RB not) (VP (VB play) (NP (NN hockey)))))
['plays', 'cricket', 'but', 'does', 'not', 'play', 'hockey']
(PRP he)
['he']
(NP (PRP he))
['he']
(S
  (NP (PRP he))
  (VP
    (VP (VBZ plays) (NP (NN cricket)))
    (CC but)
    (VP (VBZ does) (RB not) (VP (VB play) (NP (NN hockey))))))
['he', 'plays', 'cricket', 'but', 'does', 'not', 'play', 'hockey']
(ROOT
  (S
    (NP (PRP he))
    (VP
      (VP (VBZ plays) (NP (NN cricket)))
      (CC but)
      (VP (VBZ does) (RB not) (VP (VB play) (NP (NN hockey)))))))
['he', 'plays', 'cricket'

In [21]:
print(parser["sentences"][0].keys())
print(parser["sentences"][0]["parse"])
#print(sent_tree)

dict_keys(['index', 'parse', 'basicDependencies', 'enhancedDependencies', 'enhancedPlusPlusDependencies', 'tokens'])
(ROOT
  (S
    (NP
      (NP (NNP Automated))
      (VP (VBG inserting)
        (CC and)
        (VBG categorizing)
        (NP (JJ 20k) (NNS emails))
        (PP (IN from)
          (NP
            (NP
              (QP (RB over) (CD 200))
              (NNS clients))
            (PP (IN in)
              (NP (DT a) (NNP CRM)))))
        (PP (IN by)
          (S
            (VP (VBG creating)
              (NP (DT an) (NN email) (NN monitoring)))))))
    (VP
      (VP (VBD REST)
        (NP (NN API))
        (S
          (VP (VBG using)
            (NP (NNP Express)))))
      (, ,)
      (CC and)
      (VP (VBD deployed)
        (NP (DT the) (NN service))
        (PP (IN on)
          (NP
            (NP (NNP Azure) (NNP Cloud))
            (PP (IN with)
              (NP (NNP Docker)))))))
    (. .)))
